In [12]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [13]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D,GRU
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import recall_score,f1_score

In [14]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5232647576128817934
xla_global_id: -1
]


In [11]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

# Data Preprocessing for Language-English

In [15]:
train = pd.read_csv("C:/Users/lokes/Downloads/train.tsv",delimiter="\t").fillna(" ")
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate


In [17]:
train["Label"].value_counts()

Label
moderate          6004
not depression    1985
severe             902
Name: count, dtype: int64

In [18]:
train.head()

,PID,Text_data,Label
0,train_pid_1,Waiting for my mind to have a breakdown once t...,moderate
1,train_pid_2,My new years resolution : I'm gonna get my ass...,moderate
2,train_pid_3,New year : Somone else Feeling like 2020 will ...,moderate
3,train_pid_4,"My story I guess : Hi, Im from Germany and my ...",moderate
4,train_pid_5,Sat in the dark and cried myself going into th...,moderate


In [19]:
train_x = train["Text_data"].str.lower()
train_y = train["Label"]
train_y = pd.get_dummies(train_y,columns=train_y).values
train_y.shape

(8891, 3)

In [20]:
train_x.shape

(8891,)

In [21]:
max_features = 100000
maxlen = 150

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
train_x = pad_sequences(train_x,maxlen=maxlen)

In [23]:
train_x,trainxval,train_y,train_yval=train_test_split(train_x,train_y,test_size=0.2)

In [24]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


# LSTM model for Language-English (Unbalanced Data)

In [25]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = LSTM(128,return_sequences=True,dropout=0.15)(layer)
x = LSTM(128,return_sequences=True,dropout=0.15)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model2 = Model(inp,out)
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomUniformV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in

KeyboardInterrupt: 

# Bidirectional LSTM model for Language-English (Unbalanced Data)

In [18]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(layer)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
 16/223 [=>............................] - ETA: 2:20 - loss: 0.9127 - accuracy: 0.6582

KeyboardInterrupt: 

# GRU model for Language-English (Unbalanced Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = GRU(128,return_sequences=True,dropout=0.15)(layer)
x = GRU(128,return_sequences=True,dropout=0.15)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model3 = Model(inp,out)
model3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model3.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
223/223 [==============================] - 13s 47ms/step - loss: 0.7047 - accuracy: 0.7354 - val_loss: 0.5110 - val_accuracy: 0.8257
Epoch 2/5
223/223 [==============================] - 10s 46ms/step - loss: 0.3372 - accuracy: 0.8795 - val_loss: 0.3799 - val_accuracy: 0.8634
Epoch 3/5
223/223 [==============================] - 10s 44ms/step - loss: 0.1968 - accuracy: 0.9328 - val_loss: 0.4046 - val_accuracy: 0.8713
Epoch 4/5
223/223 [==============================] - 10s 43ms/step - loss: 0.1315 - accuracy: 0.9557 - val_loss: 0.4012 - val_accuracy: 0.8803
Epoch 5/5
223/223 [==============================] - 10s 43ms/step - loss: 0.1023 - accuracy: 0.9664 - val_loss: 0.4126 - val_accuracy: 0.8741


# Downsampling Data

In [ ]:
df=train[0:901]
df2=train[6019:6920]
df3=train[7990:]

df = df[['Text_data', 'Label']]
df2 = df2[['Text_data', 'Label']]
df3 = df3[['Text_data', 'Label']]

print("Moderate cases: ", str(len(df)))
print("Not Depressed cases: ", str(len(df2)))
print("Severe cases: ", str(len(df3)))

data = pd.concat([df, df2, df3], axis=0).reset_index(drop=True)

train_x = data['Text_data'].str.lower()
train_y = data['Label']
train_y = pd.get_dummies(train_y,columns=train_y).values

Moderate cases:  901
Not Depressed cases:  901
Severe cases:  901


In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
train_x = sequence.pad_sequences(train_x,maxlen=maxlen)

train_x,trainxval,train_y,train_yval=train_test_split(train_x,train_y,test_size=0.2)

# Bidirectional LSTM model for Language-English (Downsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(layer)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
68/68 [==============================] - 14s 133ms/step - loss: 0.9077 - accuracy: 0.5435 - val_loss: 0.6414 - val_accuracy: 0.7597
Epoch 2/5
68/68 [==============================] - 9s 126ms/step - loss: 0.4756 - accuracy: 0.8085 - val_loss: 0.4356 - val_accuracy: 0.8743
Epoch 3/5
68/68 [==============================] - 8s 113ms/step - loss: 0.1976 - accuracy: 0.9329 - val_loss: 0.3511 - val_accuracy: 0.9131
Epoch 4/5
68/68 [==============================] - 8s 112ms/step - loss: 0.0774 - accuracy: 0.9773 - val_loss: 0.3572 - val_accuracy: 0.9020
Epoch 5/5
68/68 [==============================] - 8s 124ms/step - loss: 0.0410 - accuracy: 0.9917 - val_loss: 0.5721 - val_accuracy: 0.9076


# LSTM model for Language-English (Downsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(layer)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
68/68 [==============================] - 13s 113ms/step - loss: 0.9244 - accuracy: 0.5463 - val_loss: 0.6859 - val_accuracy: 0.6543
Epoch 2/5
68/68 [==============================] - 6s 92ms/step - loss: 0.5003 - accuracy: 0.7854 - val_loss: 0.4309 - val_accuracy: 0.8780
Epoch 3/5
68/68 [==============================] - 6s 87ms/step - loss: 0.2022 - accuracy: 0.9371 - val_loss: 0.4947 - val_accuracy: 0.9131
Epoch 4/5
68/68 [==============================] - 6s 84ms/step - loss: 0.0945 - accuracy: 0.9709 - val_loss: 0.4793 - val_accuracy: 0.9187
Epoch 5/5
68/68 [==============================] - 5s 80ms/step - loss: 0.0415 - accuracy: 0.9884 - val_loss: 0.5520 - val_accuracy: 0.9094


# GRU model for Language-English (Downsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = GRU(128,return_sequences=True,dropout=0.15)(layer)
x = GRU(128,return_sequences=True,dropout=0.15)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model3 = Model(inp,out)
model3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model3.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
68/68 [==============================] - 5s 51ms/step - loss: 0.9485 - accuracy: 0.5116 - val_loss: 0.8038 - val_accuracy: 0.6580
Epoch 2/5
68/68 [==============================] - 3s 40ms/step - loss: 0.5305 - accuracy: 0.7900 - val_loss: 0.5919 - val_accuracy: 0.7948
Epoch 3/5
68/68 [==============================] - 3s 38ms/step - loss: 0.3095 - accuracy: 0.9033 - val_loss: 0.4702 - val_accuracy: 0.8614
Epoch 4/5
68/68 [==============================] - 3s 41ms/step - loss: 0.1794 - accuracy: 0.9408 - val_loss: 0.4514 - val_accuracy: 0.9020
Epoch 5/5
68/68 [==============================] - 3s 42ms/step - loss: 0.1023 - accuracy: 0.9718 - val_loss: 0.5450 - val_accuracy: 0.8946


# Upsampling

In [ ]:
moderate_messages = train[train["Label"] == "moderate"]
no_messages  = train[train["Label"] == "not depression"]
severe_messages=train[train["Label"] == "severe"]

print(moderate_messages.shape)
print(no_messages.shape)
print(severe_messages.shape)

(6019, 3)
(1971, 3)
(901, 3)


In [ ]:
from sklearn.utils import resample
spam_upsample_nomes = resample(no_messages,
             replace=True,
             n_samples=len(moderate_messages),
             random_state=42)

print(spam_upsample_nomes.shape)

(6019, 3)


In [ ]:
from sklearn.utils import resample
spam_upsample_severe = resample(severe_messages,
             replace=True,
             n_samples=len(moderate_messages),
             random_state=42)

print(spam_upsample_severe.shape)

(6019, 3)


In [ ]:
data = pd.concat([moderate_messages, spam_upsample_severe, spam_upsample_nomes], axis=0).reset_index(drop=True)

train_x = data['Text_data'].str.lower()
train_y = data['Label']
train_y = pd.get_dummies(train_y,columns=train_y).values

In [ ]:
data['Label'].value_counts()

moderate          6019
severe            6019
not depression    6019
Name: Label, dtype: int64

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
train_x = sequence.pad_sequences(train_x,maxlen=maxlen)

train_x,trainxval,train_y,train_yval=train_test_split(train_x,train_y,test_size=0.2)

# LSTM model for Language-English (Upsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = LSTM(128,return_sequences=True,dropout=0.15)(layer)
x = LSTM(128,return_sequences=True,dropout=0.15)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model2 = Model(inp,out)
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
452/452 [==============================] - 26s 52ms/step - loss: 0.5958 - accuracy: 0.7396 - val_loss: 0.2785 - val_accuracy: 0.8953
Epoch 2/5
452/452 [==============================] - 22s 49ms/step - loss: 0.1882 - accuracy: 0.9349 - val_loss: 0.1632 - val_accuracy: 0.9463
Epoch 3/5
452/452 [==============================] - 21s 47ms/step - loss: 0.0975 - accuracy: 0.9702 - val_loss: 0.1735 - val_accuracy: 0.9516
Epoch 4/5
452/452 [==============================] - 22s 48ms/step - loss: 0.0750 - accuracy: 0.9784 - val_loss: 0.1556 - val_accuracy: 0.9582
Epoch 5/5
452/452 [==============================] - 23s 50ms/step - loss: 0.0601 - accuracy: 0.9829 - val_loss: 0.1536 - val_accuracy: 0.9651


# BiDirectional LSTM model for Language-English (Upsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(layer)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.15))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
452/452 [==============================] - 44s 89ms/step - loss: 0.5534 - accuracy: 0.7601 - val_loss: 0.2507 - val_accuracy: 0.9147
Epoch 2/5
452/452 [==============================] - 39s 85ms/step - loss: 0.1527 - accuracy: 0.9518 - val_loss: 0.1426 - val_accuracy: 0.9607
Epoch 3/5
452/452 [==============================] - 39s 87ms/step - loss: 0.0788 - accuracy: 0.9774 - val_loss: 0.1407 - val_accuracy: 0.9618
Epoch 4/5
452/452 [==============================] - 40s 88ms/step - loss: 0.0564 - accuracy: 0.9848 - val_loss: 0.1162 - val_accuracy: 0.9687
Epoch 5/5
452/452 [==============================] - 41s 91ms/step - loss: 0.0485 - accuracy: 0.9879 - val_loss: 0.1375 - val_accuracy: 0.9693


# GRU model for Language-English (Upsampled Data)

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = GRU(128,return_sequences=True,dropout=0.15)(layer)
x = GRU(128,return_sequences=True,dropout=0.15)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(3,activation='softmax')(x)

model3 = Model(inp,out)
model3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model3.fit(train_x, train_y, batch_size=32, epochs=5, verbose=1,validation_data=[trainxval,train_yval])

Epoch 1/5
452/452 [==============================] - 24s 47ms/step - loss: 0.5778 - accuracy: 0.7423 - val_loss: 0.2786 - val_accuracy: 0.9009
Epoch 2/5
452/452 [==============================] - 19s 42ms/step - loss: 0.1775 - accuracy: 0.9432 - val_loss: 0.1727 - val_accuracy: 0.9510
Epoch 3/5
452/452 [==============================] - 18s 41ms/step - loss: 0.0961 - accuracy: 0.9717 - val_loss: 0.1591 - val_accuracy: 0.9601
Epoch 4/5
452/452 [==============================] - 19s 42ms/step - loss: 0.0679 - accuracy: 0.9808 - val_loss: 0.1450 - val_accuracy: 0.9621
Epoch 5/5
452/452 [==============================] - 19s 41ms/step - loss: 0.0563 - accuracy: 0.9834 - val_loss: 0.1422 - val_accuracy: 0.9695


# Data Preprocessing for Language-Russian

In [ ]:
train = pd.read_csv("/mnt/c/Users/lokes/Downloads/Depressive data.csv")
train.head()

,text,label,age
0,"Когда-то я был добрым романтиком, который стре...",1,32.0
1,Здраствуйте! Я каждый день просыпаюсь с мыслью...,1,28.0
2,У меня проблемы с девушкой. Каждую ссору я не ...,1,16.0
3,"Вся моя жизнь это один сплошной ад, в котором ...",1,32.0
4,Я хочу уснуть и не проснуться.каждый день одно...,1,14.0


In [ ]:
train["label"].value_counts()

0    32021
1    32018
Name: label, dtype: int64

In [ ]:
train_x = train["text"]
train_y = train["label"]
train_y = pd.get_dummies(train_y,columns=train_y).values
train_y.shape

(64039, 2)

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
train_x = sequence.pad_sequences(train_x,maxlen=maxlen)

train_x,trainxval,train_y,train_yval=train_test_split(train_x,train_y,test_size=0.2)

# LSTM model for Language-Russian

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = LSTM(128,return_sequences=True,dropout=0.4)(layer)
x = LSTM(128,return_sequences=True,dropout=0.4)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(2,activation='softmax')(x)

model2 = Model(inp,out)
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.fit(train_x, train_y, batch_size=128, epochs=1, verbose=1,validation_data=[trainxval,train_yval])

401/401 [==============================] - 25s 57ms/step - loss: 0.1073 - accuracy: 0.9547 - val_loss: 0.0251 - val_accuracy: 0.9898


# BiDirectional LSTM model for Language-Russian

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.4))(layer)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.4))(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(2,activation='softmax')(x)

model = Model(inp,out)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=128, epochs=1, verbose=1,validation_data=[trainxval,train_yval])

401/401 [==============================] - 41s 92ms/step - loss: 0.0620 - accuracy: 0.9732 - val_loss: 0.0190 - val_accuracy: 0.9936


# GRU model for Language-Russian

In [ ]:
inp = Input(name='inputs',shape=[maxlen,])

layer = Embedding(max_features,50,input_length=maxlen)(inp)
x = GRU(128,return_sequences=True,dropout=0.4)(layer)
x = GRU(128,return_sequences=True,dropout=0.4)(x)
x = Conv1D(64,kernel_size=3,padding='valid',kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool,max_pool])

out = Dense(2,activation='softmax')(x)

model3 = Model(inp,out)
model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model3.fit(train_x, train_y, batch_size=128, epochs=1, verbose=1,validation_data=[trainxval,train_yval])

401/401 [==============================] - 20s 44ms/step - loss: 0.1401 - accuracy: 0.9495 - val_loss: 0.0343 - val_accuracy: 0.9870
